In [1]:
# Standard imports
import json
from tqdm import tqdm
import sys
import pandas as pd
import random
import os
from joblib import Parallel, delayed
import joblib
from collections import defaultdict
import importlib
from functools import partial
import csv
import dotenv
import math
dotenv.load_dotenv()

True

In [2]:
# Method imports
sys.path.append('..')

from methods.mathprompterpp import mathprompterpp
from methods.mathprompter import mathprompter
from methods.zeroshotCoT import zeroshotCoT
from utils.method_utils import generating_algebraic_template
from utils.plot_utils import compute_detailed_metrics, plot_individual_metrics
from llm_inference.llm_factory import get_llm_service

import methods.mathprompterpp
import methods.mathprompter
import methods.zeroshotCoT
import utils.method_utils
import llm_inference.llm_factory

importlib.reload(methods.mathprompterpp)
importlib.reload(methods.zeroshotCoT)   
importlib.reload(utils.method_utils)
importlib.reload(llm_inference.llm_factory)


/Users/s/miniforge3/envs/llms/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'llm_inference.llm_factory' from '/Users/s/Desktop/Academics/Quarter 4/CSE256/Project/FinalCode/llm_inference/llm_factory.py'>

In [3]:
# Initialize parameters
suffix = str(random.randint(1, 1000000))
random.seed(42)
model_service = 'llama'
model_name = 'mlx-community/Llama-3.2-1B-Instruct-4bit'
# model_service = 'openai'
# model_name = 'gpt-4o-mini'
data_size = 600
max_tokens = 200
few_shot = False
consensus_k = 5

In [4]:
# Initialize model and data
# SERVICE = get_llm_service(model_service, model_name, temperature=0.0, max_tokens=max_tokens)
# SERVICE = defaultdict(lambda: None)
# def initialize_model():
#     global SERVICE
#     if SERVICE[joblib.worker_id()] is None:
#         SERVICE[joblib.worker_id()] = get_llm_service(model_service, model_name, temperature=0.0, max_tokens=max_tokens)

In [5]:
# Define the methods you want to evaluate
run_methods = ["MathPrompter", "MathPrompter++", "Zeroshot CoT"]  # Example methods

In [6]:
with open('data/multiarith.json', 'r') as file:
    data = json.load(file)
data = random.sample(data, k=data_size)  # Take a random sample of 100 rows

suffix += "_multiarith_" + model_name.split("/")[-1] + "_" + str(data_size) 
suffix += "_few_shot" if few_shot else ""
print(suffix)

# Create folder if not exists
if not os.path.exists('results/' + suffix):
    os.makedirs('results/' + suffix)

375291_multiarith_Llama-3.2-1B-Instruct-4bit_600


In [7]:
if "MathPrompter" in run_methods:
    # MathPrompter
    header_mathprompter = ["Question", "Correct Answer", "Predicted Answer", "Predicted Algebraic Answer", "Predicted Python Answer", "Generated Algebraic Template", "Generated Expression", "Generated Python Code", "Variable Mapping", "Error", "Error Message"]
    df_mathprompter = pd.DataFrame(columns=header_mathprompter)
    csv_file_mathprompter = 'results/' + suffix + '/mathprompter_results.csv'  # CSV file path

if "MathPrompter++" in run_methods:
    # MathPrompter++
    header_mathprompterpp = ["Question", "Correct Answer", "Predicted Answer", "Predicted Algebraic Answer", "Predicted Python Answer", "Generated Algebraic Template", "Rough Solution", "Generated Expression", "Generated Python Code", "Variable Mapping", "Error", "Error Message"]
    df_mathprompterpp = pd.DataFrame(columns=header_mathprompterpp)
    csv_file_mathprompterpp = 'results/' + suffix + '/mathprompterpp_results.csv'  # CSV file path

if "Zeroshot CoT" in run_methods:
    # Zeroshot CoT
    header_zeroshot_cot = ["Question", "Correct Answer", "Predicted Answer", "Response", "Error", "Error Message"]
    df_zeroshot_cot = pd.DataFrame(columns=header_zeroshot_cot)
    csv_file_zeroshot_cot = 'results/' + suffix + '/zeroshot_cot_results.csv'  # CSV file path

# Example of saving the DataFrames to CSV files
if "MathPrompter" in run_methods:
    df_mathprompter.to_csv(csv_file_mathprompter, index=False)

if "MathPrompter++" in run_methods:
    df_mathprompterpp.to_csv(csv_file_mathprompterpp, index=False)

if "Zeroshot CoT" in run_methods:
    df_zeroshot_cot.to_csv(csv_file_zeroshot_cot, index=False)


In [8]:
def reset_csv_files():
    if "MathPrompter" in run_methods:
        with open(csv_file_mathprompter, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header_mathprompter)
    if "MathPrompter++" in run_methods:
        with open(csv_file_mathprompterpp, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header_mathprompterpp)
    if "Zeroshot CoT" in run_methods:
        with open(csv_file_zeroshot_cot, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header_zeroshot_cot)


In [9]:
# Parallelize across different data rows using joblib
def process_question(item, debug = False):
    SERVICE = get_llm_service(model_service, model_name, temperature=0.0, max_tokens=max_tokens)
    question = item["question"]
    correct_answer = item["final_ans"]
    qt, variable_mapping = generating_algebraic_template(question) 
    if "MathPrompter" in run_methods:   
        mathprompter_answer = mathprompter(qt, variable_mapping, SERVICE, k = consensus_k, few_shot=few_shot)

        # Create a dictionary for the new row
        new_row = {
            "Question": question,
            "Correct Answer": correct_answer,
            "Predicted Answer": mathprompter_answer["Result"],
            "Predicted Algebraic Answer": mathprompter_answer["Expression_Result"],
            "Predicted Python Answer": mathprompter_answer["Code_Result"],
            "Generated Algebraic Template": qt,
            "Generated Expression": mathprompter_answer["Expression"],
            "Generated Python Code": mathprompter_answer["Code"],
            "Variable Mapping": variable_mapping,
            "Error": mathprompter_answer["Error"],
            "Error Message": mathprompter_answer["Error_Message"]
        }
        if debug:
            print("MathPrompter")
            print(mathprompter_answer)
            print(new_row)
        # Append the new row to the DataFrame using pd.concat
        # df_mathprompter = pd.concat([df_mathprompter, pd.DataFrame([new_row])], ignore_index=True)
        # Write to csv
        with open(csv_file_mathprompter, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row.values())
    if "MathPrompter++" in run_methods:
        mathprompterpp_answer = mathprompterpp(question, qt, variable_mapping, SERVICE, k = consensus_k, few_shot=few_shot)
        
        # Create a dictionary for the new row
        new_row = {
            "Question": question,
            "Correct Answer": correct_answer,
            "Predicted Answer": mathprompterpp_answer["Result"],
            "Predicted Algebraic Answer": mathprompterpp_answer["Expression_Result"],
            "Predicted Python Answer": mathprompterpp_answer["Code_Result"],
            "Generated Algebraic Template": qt,
            "Rough Solution": mathprompterpp_answer["Rough_Solution"],
            "Generated Expression": mathprompterpp_answer["Expression"],
            "Generated Python Code": mathprompterpp_answer["Code"],
            "Variable Mapping": variable_mapping,
            "Error": mathprompterpp_answer["Error"],
            "Error Message": mathprompterpp_answer["Error_Message"]
        }
        if debug:
            print("MathPrompter++")
            print(mathprompterpp_answer)
            print(new_row)
        # Append the new row to the DataFrame using pd.concat
        # df_mathprompterpp = pd.concat([df_mathprompterpp, pd.DataFrame([new_row])], ignore_index=True)
        # Write to csv
        with open(csv_file_mathprompterpp, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row.values())
    if "Zeroshot CoT" in run_methods:
        zeroshot_cot_answer = zeroshotCoT(question, SERVICE, max_tokens)    

        # Create a dictionary for the new row
        new_row = {
            "Question": question,
            "Correct Answer": correct_answer,
            "Predicted Answer": zeroshot_cot_answer["Answer"],
            "Response": zeroshot_cot_answer["Response"],
            "Error": zeroshot_cot_answer["Error"],
            "Error Message": zeroshot_cot_answer["Error_Message"]
        }
        if debug:
            print("Zeroshot CoT")
            print(zeroshot_cot_answer)
            print(new_row)
        # Append the new row to the DataFrame using pd.concat
        # df_zeroshot_cot = pd.concat([df_zeroshot_cot, pd.DataFrame([new_row])], ignore_index=True)
        # Write to csv
        with open(csv_file_zeroshot_cot, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row.values())

In [10]:
# Test 
# reset_csv_files()
# # run_methods = ["MathPrompter", "MathPrompter++"]
# for i in tqdm(range(600)):
#     # print(data[i])
#     process_question(data[i], debug=False)

In [11]:
# Run the parallelized function
reset_csv_files()
run_methods = ["MathPrompter", "MathPrompter++"]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
process_question_partial = partial(process_question, debug=False)
Parallel(n_jobs=4)(delayed(process_question_partial)(item) for item in tqdm(data[:100], desc="Processing Questions"))


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 12501.65it/s]


Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType


Processing Questions:   8%|▊         | 8/100 [00:08<01:34,  1.03s/it]

-3
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
An error occurred during the verification process: 'Code_Result'
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
214
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 16810.84it/s]


...

Syntax error in the expression: invalid syntax (<string>, line 1)
An error occurred during the verification process: must be real number, not NoneType
An error occurred during the verification process: 'Code_Result'


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 6431.34it/s]


Syntax error in the expression: invalid syntax (<string>, line 1)
An error occurred during the verification process: must be real number, not NoneType
An error occurred during the verification process: 'Code_Result'


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 7275.46it/s]


3
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
An error occurred during the verification process: 'Code_Result'
